In [1]:
%matplotlib inline
import d2lzh as d2l
from mxnet import autograd, nd

In [2]:
# 获取和读取数据

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [3]:
# 初始化模型参数

num_inputs = 784
num_outputs = 10

W = nd.random.normal(scale=0.01, shape=(num_inputs, num_outputs))
b = nd.zeros(num_outputs)

W.attach_grad()
b.attach_grad()

In [18]:
#实现softmax计算

X = nd.array([[1, 2, 3], [4, 5, 6]])
print(X.sum(axis=0, keepdims=True), X.sum(axis=1, keepdims=True))

def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(axis=1, keepdims=True)
    return X_exp / partition
    
X = nd.random.normal(shape=(2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(axis = 1)


[[ 5.  7.  9.]]
<NDArray 1x3 @cpu(0)> 
[[  6.]
 [ 15.]]
<NDArray 2x1 @cpu(0)>


(
 [[ 0.42250919  0.1234768   0.21075098  0.02119259  0.22207047]
  [ 0.05703577  0.43163839  0.04676707  0.20360941  0.26094934]]
 <NDArray 2x5 @cpu(0)>, 
 [ 1.  1.]
 <NDArray 2 @cpu(0)>)

In [7]:
# 定义模型

def net(X):
    return softmax(nd.dot(X.reshape(-1, num_inputs), W) + b)

In [19]:
# 定义损失函数

y_hat = nd.array([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = nd.array([0, 2], dtype='int32')
nd.pick(y_hat, y)

def cross_entrop(y_hat, y):
    return - nd.pick(y_hat, y).log()

In [23]:
# 计算准确率
def accuracy(y_hat, y):
    return (y_hat.argmax(axis=1) == y.astype('float32')).mean().asscalar()

accuracy(y_hat, y)

def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        y = y.astype('float32')
        acc_sum += (net(X).argmax(axis=1) == y).sum().asscalar()
        n += y.size
    return acc_sum / n

evaluate_accuracy(test_iter, net)

0.085599999999999996

In [29]:
#训练模型

num_epochs, lr = 5, 0.1

def train_ch3(net, train_iter, loss, num_epochs, batch_size,
             params=None, lr=None, trainer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n=0.0, 0.0, 0
        for X, y in train_iter:
            with autograd.grad():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            if trainer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscaler()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscaler()
            n += y.size()
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
             % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))



train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)

epoch 1, loss 0.7899, train acc 0.745, test acc 0.801
epoch 2, loss 0.5733, train acc 0.812, test acc 0.819


KeyboardInterrupt: 